### PROGETTO FINALE IDS 


Utilizzeremo un dataset della stagione di calcio della Serie A 2023/2024, per provare a predire le ultime 5 giornate che vanno dalla 33° giornata alla 38°giornata.
In un campionato con 20 squadre e un totale di 38 giornate, ogni squadra gioca 38 partite (una per giornata).

Vogliamo considerare le ultime 5 giornate del campionato, quindi 50 partite in totale (10 partite per giornata × 5 giornate).

Per ciascuna partita, proveremo a prevedere il risultato usando uno dei seguenti segni:

1: vittoria della squadra di casa

X: pareggio

2: vittoria della squadra in trasferta

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

Importiamo la tabella Serie A_matches_23_24.csv che contiene i risultati di tutte le partite del campionato e guardiamo cosa contiene con .head

In [33]:
matches = pd.read_csv('Serie A_matches_23_24.csv')
matches.head()


,Round,Round Name,Home Team,Away Team,UTC Time,Finished,Started,Cancelled,Awarded,Score,Match Status,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1,1,Empoli,Hellas Verona,2023-08-19T16:30:00Z,True,True,False,False,0_1,Full-Time,NaN,NaN,NaN,NaN
1,1,1,Frosinone,Napoli,2023-08-19T16:30:00Z,True,True,False,False,1_3,Full-Time,NaN,NaN,NaN,NaN
2,1,1,Genoa,Fiorentina,2023-08-19T18:45:00Z,True,True,False,False,1_4,Full-Time,NaN,NaN,NaN,NaN
3,1,1,Inter,Monza,2023-08-19T18:45:00Z,True,True,False,False,2_0,Full-Time,NaN,NaN,NaN,NaN
4,1,1,Roma,Salernitana,2023-08-20T16:30:00Z,True,True,False,False,2_2,Full-Time,NaN,NaN,NaN,NaN


Come si puo notare dalla tabella, Round e roundname indicano la stessa cosa.
Ci sono delle colonne vuote e le eliminiamo.

In [34]:
drop_columns = ['Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14','Round Name']
matches.drop(drop_columns, axis=1, inplace=True)


In [35]:
matches.head()

,Round,Home Team,Away Team,UTC Time,Finished,Started,Cancelled,Awarded,Score,Match Status
0,1,Empoli,Hellas Verona,2023-08-19T16:30:00Z,True,True,False,False,0_1,Full-Time
1,1,Frosinone,Napoli,2023-08-19T16:30:00Z,True,True,False,False,1_3,Full-Time
2,1,Genoa,Fiorentina,2023-08-19T18:45:00Z,True,True,False,False,1_4,Full-Time
3,1,Inter,Monza,2023-08-19T18:45:00Z,True,True,False,False,2_0,Full-Time
4,1,Roma,Salernitana,2023-08-20T16:30:00Z,True,True,False,False,2_2,Full-Time


In [36]:
#controlliamo se ci sono valori nulli
matches.isnull().sum()

Round           0
Home Team       0
Away Team       0
UTC Time        0
Finished        0
Started         0
Cancelled       0
Awarded         0
Score           0
Match Status    0
dtype: int64

Ci siamo accorti che la colonna Score che indica come è andato a finire il risultato di un match contiene il valore del risultato in una singola colonna, cerchiamo di dividere in 2 colonne separate questi valori salvandoli in Score_casa e Score_trasferta

In [37]:
matches[['Score_casa', 'Score_trasferta']] = matches['Score'].str.split('_', expand=True).astype(int)
matches.drop('Score', axis=1, inplace=True)
matches.head()

,Round,Home Team,Away Team,UTC Time,Finished,Started,Cancelled,Awarded,Match Status,Score_casa,Score_trasferta
0,1,Empoli,Hellas Verona,2023-08-19T16:30:00Z,True,True,False,False,Full-Time,0,1
1,1,Frosinone,Napoli,2023-08-19T16:30:00Z,True,True,False,False,Full-Time,1,3
2,1,Genoa,Fiorentina,2023-08-19T18:45:00Z,True,True,False,False,Full-Time,1,4
3,1,Inter,Monza,2023-08-19T18:45:00Z,True,True,False,False,Full-Time,2,0
4,1,Roma,Salernitana,2023-08-20T16:30:00Z,True,True,False,False,Full-Time,2,2


Ora creiamo una nuova colonna "Risultato" che va a confrontare i valori di score casa e score trasferta e li inserisce nella nuova colonna.

Se score_casa e > di score_trasferta allora risultato = 1.
Se score_casa e < di score_trasferta allora risultato = 2.
Se score_casa e = a score_trasferta allora risultato = X.

In [40]:
#Se score_casa e > di score_trasferta allora risultato = 1.
#Se score_casa e < di score_trasferta allora risultato = 2.
#Se score_casa e = a score_trasferta allora risultato = X.
matches['Risultato'] = matches[np.where(matches['Score_casa'] > matches['Score_trasferta'], 1)]
matches.head()

ValueError: either both or neither of x and y should be given